## INTRODUCTION
  In (Orellana et al. 2016) we have used all the crystal structures of a protein to create an ensemble. What we call an ensemble in this article is a set of structures that are not chimeric, represent the whole protein rather than a single domain of it, and do not contain more than 5 consecutive non-terminal missing residues. As much as we would like to automate this process, preparation of an ensemble often requires manual intervention due to problems like, chimeric constructs or the placement of subunits clockwise vs. anticlockwise. There is also the decision to be made whether to exclude a structure that has missing residues that are not terminal. 
  
  While preparing the ensembles for the article, we have taken care of those problems manually and made a decision to repair the structures  if it was known to represent an important state which does not have any other representatives. Altough it very much depends on the structure, whether there are other structures in a similar state to be used as template or where the missing residues are located, we recommend repairing structures with less than 6 consecutive residues missing. There is an example modeller script that can be used to repair structures.  It is upto the user to fix and include them in the ensemble later.
  
  This tutorial will be using GLIC as an example. Because this was most challenging ensemble we prepared for this study, it highlights the potential issues the users likely to encounter. 

## HOW TO

### Requirements
  Before we get started make sure you have installed the pdbParser package and the other required python packages. If you have used pip, it should take care of the requirements for you. But if you run into problems you can find the information on the github page.
  
  To determine the missing residues, pdbParser relies on sequence alignment. If you use pdbParser to prepare a start and an end structure for eBDIMs then the alignment is generated via BioPython. However if you are preparing an ensemble you must install Clustal Omega or run the provided fasta sequence through a multiple sequence alignment tool and save the alignment in fasta format. The installation of Clustal Omega can easily be done via conda. After you install it you should provide the path for the clustal executable, if it is not in your path already.

  Altough not a requirement, if you want to fix any broken structures you will need a modelling program. We have provided an example script for modeller.

### Obtaining PDB Files
  It can be daunting to find all the available PDB files, and which chains actually belongs to your protein, which are not just antibodies or other things. The easiset way to do this is to go through the UniProtKB database and this tutorial requires the UniProt ID. ![title](img/uniprot_ini_search.png)
After finding the UniProt ID you can run the commands below to download the PDB file. Besides the UniProt ID, you also have to provide the total number of chains that are in a biological assembly. Heteromeric structures require a different routine which are not yet supported in pdbParser. However similar logic applies in preperation of such an ensemble.

  To download these structures in a different location, you can also provide an argument with an absolute path. Otherwise the commands will the current working directory for downloading files and writing outputs.

In [28]:
query='Q7NDN8'
mer=5

In [29]:
from pdbParser import prepENS as pE
info=pE.PDBInfo(query,mer)


Q7NDN8


CRITICAL:root:PDB ID 4YEU is a chimera, skipping this file
CRITICAL:root:PDB ID 5OSA is a chimera, skipping this file
CRITICAL:root:PDB ID 5OSC is a chimera, skipping this file
CRITICAL:root:PDB ID 5OSB is a chimera, skipping this file
CRITICAL:root:PDB ID 4X5T is a chimera, skipping this file


  What this PDBInfo function does is that it goes into the UniProt page of the given ID, and retrive the 3D structures table (as seen in figure below) and the reference sequence to be used for the alignment. For the reasons below it is much easier to prepare the ensemble via UniProtKB:
  
* Because there is a possiblity of all the structures having the same missing residues, the reference sequence is a good control to have. 
* Within 3D structure table, only the ones that has an "X-ray" label are downloaded. The relevant chains for this protein is also extracted from this table. In the example above, you already see that 3IGQ structure is skipped. That is because this file had 6 subunits vs. 5. If you go into the RCSB Database and looked at this structure you see that it contains only the extracellular domain and not the whole structure.

![title](img/uniprot_3dtable.png)

  After the information is gathered, info object is populated with the possible structures to be used and the reference sequence. You can reach the list via commands below. If you want to manipulate this list, add or remove structures, or change the reference sequence you can do so at this stage.

In [ ]:
info.result #Contains the information from the 3D structures table.
info.refseq #Contains the reference sequence. 

  The next command downloads the structures from the RCSB database. Depending on the number of structures this function might take a while to finish because after it downloads the files it goes through the steps below:
  
  1. It seperates the biological assemblies within the same PDB file
  2. It keeps only the CA coordinates, which are the only relevant atoms for comparison with eBDIMs trajectories. Obtaining CA coordinates only also cleans the PDB files from bound ligands, ions, waters, etc.
  3. It extracts the sequence and the residue numbering from the CA coordinates and writes them to text files.

The residue numbering does not have to be continues since this script relies on the sequence alignment to determine the missing residues. This also allows us to ignore the differences in residue numbering between different structures. The residue numbering is only used to extract the residues. Another important aspect is that we extract the sequence from the CA coordinates because despite being stated in the sequence a residue might have missing CA coordinate.

  In the example below you see that there are more warnings, stating that more structures will be excluded from the ensemble because they are chimeras. The function checks for two keywords in the title 'CHIMERA' and 'FUSED'. Since these structures cannot be used, their information is removed from the info.result dictionary automatically and those files are deleted.

In [ ]:
pE.downloadPDB(info,query,cwd)

  If you check the output folder, you will find two text files. "_seq.txt" and "_residmap.txt" We need the information within these files for the multiple sequence alignment and the subsequent cleaning of the terminal residues that are not shared between the structures. It is possible that some structures still contain a terminal tag, that would interfere with the alignment and introduce gaps unncessary gaps. Any chain that contains a non-terminal gap at this stage is marked as broken and the assembly they belong to is skipped. As you see there are quite a few chains with missing residues or structures that has insertions.
  All these identified broken chains are stored in info.broken object. This list is used in the subsequent function that extracts the shared CA region.

In [30]:
clustalopath="/Users/cattibrie_fr/anaconda/anaconda/bin/clustalo"
pE.msa(query+'_seq.txt',query+'_residmap.txt',query,cwd,clustalopath,info.result,info.mer)

CRITICAL:root:4NPQ_2.pdb|H| contains insertions or missing residues, skipping this chain and the assembly it belongs to.
CRITICAL:root:4NPQ_2.pdb|I| contains insertions or missing residues, skipping this chain and the assembly it belongs to.
CRITICAL:root:4NPQ_3.pdb|K| contains insertions or missing residues, skipping this chain and the assembly it belongs to.
CRITICAL:root:4NPQ_3.pdb|L| contains insertions or missing residues, skipping this chain and the assembly it belongs to.
CRITICAL:root:4NPQ_3.pdb|M| contains insertions or missing residues, skipping this chain and the assembly it belongs to.
CRITICAL:root:4NPQ_4.pdb|S| contains insertions or missing residues, skipping this chain and the assembly it belongs to.
CRITICAL:root:3UU3_1.pdb|A| contains insertions or missing residues, skipping this chain and the assembly it belongs to.
CRITICAL:root:3UU3_1.pdb|B| contains insertions or missing residues, skipping this chain and the assembly it belongs to.
CRITICAL:root:3UU3_1.pdb|C| cont

{'6F0R_1.pdb': ['A', 'B', 'C', 'D', 'E'], '2XQ6_1.pdb': ['A', 'B', 'C', 'D', 'E'], '4HFI_1.pdb': ['A', 'B', 'C', 'D', 'E'], '6F0M_1.pdb': ['A', 'B', 'C', 'D', 'E'], '4LML_1.pdb': ['A', 'B', 'C', 'D', 'E'], '2XQ8_1.pdb': ['A', 'B', 'C', 'D', 'E'], '3UU8_1.pdb': ['A', 'B', 'C', 'D', 'E'], '6EMX_1.pdb': ['A', 'B', 'C', 'D', 'E'], '6F15_1.pdb': ['A', 'B', 'C', 'D', 'E'], '4HFB_1.pdb': ['A', 'B', 'C', 'D', 'E'], '6F11_1.pdb': ['A', 'B', 'C', 'D', 'E'], '5MVN_1.pdb': ['A', 'B', 'C', 'D', 'E'], '3EAM_1.pdb': ['A', 'B', 'C', 'D', 'E'], '5MZQ_1.pdb': ['A', 'B', 'C', 'D', 'E'], '4ILC_1.pdb': ['A', 'B', 'C', 'D', 'E'], '5IUX_1.pdb': ['A', 'B', 'C', 'D', 'E'], '2XQ5_1.pdb': ['A', 'B', 'C', 'D', 'E'], '6F10_1.pdb': ['A', 'B', 'C', 'D', 'E'], '4HFH_1.pdb': ['A', 'B', 'C', 'D', 'E'], '4ILB_1.pdb': ['A', 'B', 'C', 'D', 'E'], '4IL9_1.pdb': ['A', 'B', 'C', 'D', 'E'], '5MVM_1.pdb': ['A', 'B', 'C', 'D', 'E'], '4LMK_1.pdb': ['A', 'B', 'C', 'D', 'E'], '4QH4_1.pdb': ['A', 'B', 'C', 'D', 'E'], '6F0J_1.pdb': [

  Altough these assemblies are skipped, the CA coordinates and the FASTA alignment information is kept. For example from the fasta alignment file, we saw that chain S in 4NPQ PDB file, assembly 4 has two missing residues. But the rest of the structures in this assembly is intact. Therefor in the following steps we fixed this structure by rebuilding those missing residues using the example modeller scripts and modify the info.broken object to exclude that chain.